#### Import library and setup notebook

In [1]:
import pandas as pd
import re
import unicodedata
import matplotlib.pyplot as plt

# Show wider options
pd.set_option('display.max_colwidth', 150)

#### Clean data, standardize Vietnamese (remove accents, stop words)

##### Text cleaning function

In [5]:
def clean_text(text):
    text = str(text).lower()  # convert to lowercase
    text = re.sub(r"http\S+|www\S+|https\S+", '', text)  # remove link
    text = re.sub(r"[^a-zA-Z0-9àáạảãâầấậẩẫăằắặẳẵèéẹẻẽêềếệểễ"
                  r"ìíịỉĩòóọỏõôồốộổỗơờớợởỡùúụủũưừứựửữỳýỵỷỹđ\s]", '', text)  # remove special characters
    text = re.sub(r"\s+", " ", text).strip()  # remove extra spaces
    return text

1. **Chuyển thành chữ thường:** giúp thống nhất dữ liệu (`"Tốt"` → `"tốt"`).  
2. **Xóa liên kết (URL):** loại bỏ các đường dẫn web (`https://...`).  
3. **Xóa ký tự đặc biệt:** chỉ giữ lại chữ, số và ký tự tiếng Việt.  
4. **Xóa khoảng trắng thừa:** gom nhiều khoảng trắng thành một, xóa đầu/cuối.

##### Vietnamese accent removal function

In [6]:
def remove_vietnamese_accents(text):
    # Split letters and accents into separate parts (e.g. 'ệ' -> 'e' and '̣')
    nfkd_form = unicodedata.normalize('NFKD', text)

    # Keep the non-accent characters, then concatenate them
    return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

1. **Chuẩn hóa ký tự:** tách phần chữ và phần dấu (`'ệ' → 'e' + '̣'`).
2. **Loại bỏ dấu:** giữ lại ký tự gốc, bỏ phần dấu (`'đẹp quá' → 'dep qua'`).

##### Remove stopwords function

In [7]:
def remove_stopwords(text, stopwords):
    words = text.split()  # Split text into list words.
    
    filtered = [w for w in words if w not in stopwords]  # Filter words that are not stopwords.
    
    return " ".join(filtered)  # Concatenate filtered to string

1. **Tách câu thành danh sách từ:** (`"tôi thích uống trà"` → `["tôi", "thích", "uống", "trà"]`)
2. **Lọc bỏ từ dừng:** giữ lại những từ không nằm trong danh sách stopwords.
3. **Ghép lại thành chuỗi:** (`["thích", "uống", "trà"]` → `"thích uống trà"`)

##### Download the list of stop words

In [8]:
stopwords = set(open("D:/Projects/Sentiment_Analysis/data/external/vietnamese-stopwords.txt", "r", encoding="utf-8").read().splitlines())

#### Read and Process data

In [9]:
# Read each episode
train_df = pd.read_csv("D:/Projects/Sentiment_Analysis/data/interim/train.csv")
test_df  = pd.read_csv("D:/Projects/Sentiment_Analysis/data/interim/test.csv")
dev_df   = pd.read_csv("D:/Projects/Sentiment_Analysis/data/interim/dev.csv")

# Apply the same handler function to all three
for df in [train_df, test_df, dev_df]:
    df["sentence"] = df["sentence"].apply(clean_text)
    df["sentence"] = df["sentence"].apply(remove_vietnamese_accents)
    df["sentence"] = df["sentence"].apply(lambda x: remove_stopwords(x, stopwords))

# Save
train_df.to_csv("D:/Projects/Sentiment_Analysis/data/processed/train_clean.csv", index=False)
test_df.to_csv("D:/Projects/Sentiment_Analysis/data/processed/test_clean.csv", index=False)
dev_df.to_csv("D:/Projects/Sentiment_Analysis/data/processed/dev_clean.csv", index=False)